In [2]:
import random
import numpy as np
from modestga import minimize
from modestga import con_minimize

In [3]:
import numpy as np
import time
import copy
import random
from importlib import reload
import pandas as pd

In [4]:
import __init__
import distributions
import kernel
import fuzzifier
import optimization

reload(__init__)
reload(distributions)
reload(kernel)
reload(fuzzifier)
reload(optimization)

from __init__ import *
from distributions import *
from kernel import *
from fuzzifier import *
from optimization import *

tensorflow not available
tensorflow not available


In [5]:
def generate_dataset(n_samples=10, n_features=1):

    #random.seed(42)
    
    X = np.zeros((n_samples, n_features))

    for elem in range(n_samples):
        for dim in range(n_features):
            X[elem][dim] = random.uniform(0,1)

    gaussian = lambda x,var : (np.e ** (-(((x-0.5)*2)**2)/(2*var)))

    y = np.zeros(n_samples)
    
    for elem in range(n_samples):
        y[elem] = 1
        for dim in range(n_features):
            y[elem] *= gaussian(X[elem][dim], np.var([X[j][dim] for j in range(n_samples)]))
                          
    return X,y

In [10]:
X, mus = generate_dataset(1000)
C = 1

kern = kernel.GaussianKernel()
s = time.time()
k = np.array([kern.compute(x1, X) for x1 in X])
print('time: {:.2f}s'.format(time.time() - s))

time: 0.08s


In [11]:
print("gurobi start")
s = time.time()
solver = GurobiSolver(adjustment='auto')
res_gurobi = np.array(solver.solve(X, mus, C, k))
time_gurobi = time.time() - s

def fun(x, k):
    return -x.dot(np.diag(k,0)) + x.dot(k.dot(x)) 
bounds = [(-C*(1-mu) , (C*mu)) for mu in mus]
def constr1(x, k):
    return np.sum(x) - 1  
def constr2(x, k):
    return -np.sum(x) + 1 
constr = (constr1, constr2)
options = {"generations": 1000, "pop_size": 1600, "tol": 1e-2, 
           "mut_rate": 0.05, "xover_ratio": 0.5, "trm_size": 20}
print("genetic start")
s = time.time()
res = con_minimize(fun, bounds, constr=constr, args=[k], options=options, workers=10)
time_genetic = time.time() - s
res_genetic = res.x

gurobi start
genetic start


In [12]:
print('dataset shape: ', X.shape)
print()

print("genetic result: ", fun(res_genetic,k))
print("        constraints: ", constr1(res_genetic,k), constr2(res_genetic,k))
print('        time: {:.2f}s'.format(time_genetic))
print('        message: ', res.message)

print()

print("gurobi result: ", fun(res_gurobi,k))
print("       constraints: ", constr1(res_gurobi,k), constr2(res_gurobi,k))
print('       time: {:.2f}s'.format(time_gurobi))

dataset shape:  (1000, 1)

genetic result:  -0.3800023907417176
        constraints:  -0.24675099241530596 0.24675099241530596
        time: 217.23s
        message:  Solution improvement below tolerance for 100 generations

gurobi result:  -0.377312850519139
       constraints:  -5.1958437552457326e-14 5.1958437552457326e-14
       time: 2.86s


In [13]:
len(mus) - np.sum([-C*(1-mu) <= x <= (C*mu) for x,mu in zip(res_genetic,mus)])

28

In [14]:
len(mus) - np.sum([-C*(1-mu) <= x <= (C*mu) for x,mu in zip(res_gurobi,mus)])

0